# Homework 3

DUE Nov 15th at 11:59 PM

## Problem 1

In this problem, you will implement a simple feed-forward neural network using PyTorch, a straight-forward and simple-to-pickup framework for quickly prototyping deep learning model. 

PyTorch provides 2 powerful things. First, a nice data structure called Tensor (basically a matrix, similar to Numpy ndarray). Tensor is optimized for matrix calculation and can be loaded to a GPU. Tensor is also implemented so that it's easy to calculate and pass back chains of gradients, which is extremely useful for backpropagation on neural network. Second, a nice inner mechanism called Autograd that nicely maps variables involved a chain of calculations and efficiently calculates their gradients via the chain rule when needed. Read more here: https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95  

You will define a neural network class in PyTorch and use the network to learn a classification task on the famous KDD CUP 99 dataset. You can refer to Problem 2 to see how a network class can be defined, how to use a PyTorch's DataLoader, and how a training loop may looks like.

There are many greate tutorial on PyTorch out there. For example, this video on Youtube explains how to build a simple network in PyTorch quite clearly: https://www.youtube.com/watch?v=oPhxf2fXHkQ

### Part a
Firstly, load and inspect the "**KDD CUP 99**" dataset.

In [1]:
from sklearn.datasets import fetch_kddcup99

X, y = fetch_kddcup99(return_X_y=True, percent10=True)

Split them into a train set (70%), a validation set (20%), and a test set (20%). Then, create a PyTorch's DataLoader for the train set, a DataLoader for the validation set, and a DataLoader for the test set.

You can read about PyTorch's DataLoader from:

*   https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
*   https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader



In [4]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Your answer goes here

print(X)

# As you can see, we have some categorical values for a few of our rows

print(X[:, 1])

# We can fix this by going back to the Data Cleaning demo I showed in week 1
# Essentially, we can use a label encoder to change the categorical values into numerical ones

[[0 b'tcp' b'http' ... 0.0 0.0 0.0]
 [0 b'tcp' b'http' ... 0.0 0.0 0.0]
 [0 b'tcp' b'http' ... 0.0 0.0 0.0]
 ...
 [0 b'tcp' b'http' ... 0.01 0.0 0.0]
 [0 b'tcp' b'http' ... 0.01 0.0 0.0]
 [0 b'tcp' b'http' ... 0.01 0.0 0.0]]
[b'tcp' b'tcp' b'tcp' ... b'tcp' b'tcp' b'tcp']


In [8]:
# Reminder of how to use a label encoder
from sklearn import preprocessing
import numpy as np

myString = "hereishowyoucanusealabelencoder"
dummyArray = []
for char in myString:
  dummyArray.append(char)
print(dummyArray)

chars = preprocessing.LabelEncoder()
dummyArray = chars.fit_transform(dummyArray)
print(dummyArray)

['h', 'e', 'r', 'e', 'i', 's', 'h', 'o', 'w', 'y', 'o', 'u', 'c', 'a', 'n', 'u', 's', 'e', 'a', 'l', 'a', 'b', 'e', 'l', 'e', 'n', 'c', 'o', 'd', 'e', 'r']
[ 5  4 10  4  6 11  5  9 13 14  9 12  2  0  8 12 11  4  0  7  0  1  4  7
  4  8  2  9  3  4 10]


To instantiate a dataloader into our model, let's take a look at the website that was provided that walks through the dataloaders

Yingrui also gave a good implementation of it in her discussion section last week. This implementation can be seen below

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, X, y):
        'Initialization'
        self.labels = torch.tensor(y)
        self.features = torch.tensor(X)

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.labels)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample

        return self.features[index], self.labels[index]

# After we train, validate, split our data, we can just create Datasets
# from our dataset class and use those to create Dataloaders

# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
# If we look at the Dataloader class, we can look at how to create them and what
# parameters we need

### Example Code from Question 2

To solve problems 1 and 2, we can look at the example code from question 2 as a guideline for what we are doing. To be able to adapt our code for problem 2, let's figure out how each of these things are working

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Here we are creating a class for the MNIST dataset
class mnist_network(torch.nn.Module):
    
    def __init__(self, num_hidden_layers=1, layer_size = 100, activation=None):
        super(mnist_network, self).__init__()
        # In the initialization function, we are defining the parameters given
        # to us from our function call and storing these values within our model
        self.num_hidden_layers = num_hidden_layers
        self.layer_size = layer_size
        self.activation = activation

        if(self.activation is 'relu'):
            self.activation = F.relu
        elif(self.activation is 'tanh'):
            self.activation = torch.tanh

        # This line of code looks at the first layer in our model, which is the
        # input model. The size of the input for our neural network is the number
        # of features our data has
        # 784 is specific to the MNIST dataset, and will not work for KDD CUP 99
        self.layers = nn.ModuleList([nn.Linear(784,self.layer_size)])

        # This next loop is only necessary if you have more than 1 hidden layer
        # but works regardless of how many hidden layers you have
        # What this does is creates a loop and maps each layer size to another
        # hidden layer of that size
        for i in range(1, self.num_hidden_layers):
            self.layers.append(nn.Linear(self.layer_size,self.layer_size))

        # And this last layer is the output layer, which is the number of classes
        # we have for our dataset. Again, 10 is specific to the number of output
        # classes for MNIST, you will have to change this for problem 1
        self.layers.append(nn.Linear(self.layer_size,10))

        # As we see, the initialization function created all our layers and 
        # made them the correct size

    # Now we can define a forward function for when we are moving forward through
    # the layers of our network
    def forward(self, x):

        # converting each image into a vector
        # Your input vectors will be the wrong size without these two lines
        batch_size = x.shape[0]
        x = x.reshape(batch_size,-1)

        # Now we loop through all our layers and use the activation method we 
        # input into our model
        for i in range(self.num_hidden_layers+1):
            x = self.layers[i](x)
            if(self.activation is not None):
                x = self.activation(x)
        return x

In [ ]:
import torch
from torchvision import datasets, transforms

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)

# DataLoader is a nice tool provided by PyTorch for passing training or testing examples
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1000)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import itertools

# Here we are creating two methods to train and test our model 

def train(model, criterion, train_loader, optimizer, epoch):
    # Turn the model to training mode (gradients will be calculated)
    model.train()
    # Loop through our dataloader
    for batch_idx, (data, target) in enumerate(train_loader):
        # We have to call zero_grad() on the optimizer to remove gradients from the previous data pass.
        # Otherwise, the gradients will be accumulated throughout many passes.
        optimizer.zero_grad()
        # Pass in the data and obtain the output.
        # When you pass the data directly by calling model(data), the model will internally pass the data through the forward() function.
        output = model(data)
        # Compare the output and the ground truth and calculate the loss.
        loss = criterion(output, target)
        # From the calculated loss, call backward() to calculate the gradients for all the paramters in the network.
        loss.backward()
        # Update the parameters according to the gradients. 
        optimizer.step()
        
        # This is to print out the progress you see when you run your code
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, criterion, test_loader):
    # Turn the model to testing mode (gradients will not be calculated)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        # Loop through our data in the dataloader
        for data, target in test_loader:
            output = model(data)
            # sum up batch loss
            test_loss += criterion(output, target).item()
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True) 
            # sum up the correct number of items that you have guessed
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
epochs = 5
lr = 0.01

model = mnist_network(num_hidden_layers=1,layer_size=20,activation='relu')

# Define the training and testing loss as cross entropy
train_criterion = nn.CrossEntropyLoss()
test_criterion = nn.CrossEntropyLoss(reduction='sum')

# Define the optimizer
# We have to specify the learning rate and the parameters that the optimizer should update during training.
# In this case, we specify that the optimizer should update all the parameters from our model.
optimizer = optim.SGD(model.parameters(),lr=lr)

# Now for every epoch, it will go through and train and test our model
for epoch in range(1, epochs + 1):
        # Training
        train(model, train_criterion, train_loader, optimizer, epoch)
        # Testing
        test(model, test_criterion, test_loader)

Once you define your model classes and dataloaders, these last 2 cells should be relatively the same, as training, testing, and looping through the epochs should not change much from model to model.

### Part b 
Create a Python class for our neural network model. The network should have 1 input layer, 1 hidden layer, and 1 output layer. You are free to choose the size of the hidden layer (it may affect the performance). Use ReLU as the activation function (torch.relu).

In [ ]:
import torch

# Any Pytorch's network class is an extension of the torch.nn.Module parent class.
# To define a network class, you need to define at least 2 methods: an __init__() method (constructor) and a forward() method
class SimpleNetwork(torch.nn.Module):
    # Create the network class by filling in this block of code

    # Create the constructor. Add any additional arguments as you wish
    def __init__(self):
        pass

    # Define the feed forward function.
    # x is the input example/examples.
    # Add any additional arguments as you wish.
    def forward(self, x):
        pass

### Part c 
Train the network using the training dataset. Use the SGD optimizer and CrossEntropyLoss. After each epoch, record the current loss and the current training accuracy. The current training accuracy is obtained by evaluating the model on the train dataset. Use the DataLoaders defined in part a to efficiently pass training and testing data.

You can learn about the available optimizers at:
https://pytorch.org/docs/stable/optim.html

You can learn about the available loss functions at:
https://pytorch.org/docs/stable/nn.html#loss-functions

In [ ]:
LEARNING_RATE = 0.01
EPOCHS = 100

for epoch in range(EPOCHS):
    # Train the network by filling in this block of code

Plot how the loss and the training accuracy and the validation accuracy change over the epochs. Is there a point where overfitting occurs? If you cannot spot one, answer no. 

### Part d 
Evaluate the model on the test dataset. Print out the accuracy. Does this accuracy agrees with the training accuracy showed on the plot?

## Problem 2

In this problem, we will investigate the effects of various common hyperparameters on the performance of a neural network. In the following cell, you can find a network class already defined for you. You can initiate network instances with different hyperparameters by changing the contructor's arguments.

You are graded based on how you implement and execute the experiments. Since there is some randomness in initiating and training a neural network, there is no guarantee that you will get an expected result for an experiment or that your results should be similar to those of your peers. The expected outcome is that you execute the experiments correctly and the conclusion you get are consistent with your results. For each experiments, try to run the code multiple times and record the average results like what we did in Homework 2 (it will take some time to run, as expected when training any neural network).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class mnist_network(torch.nn.Module):
    
    def __init__(self, num_hidden_layers=1, layer_size = 100, activation=None):
        super(mnist_network, self).__init__()
        # layers of the network
        self.num_hidden_layers = num_hidden_layers
        self.layer_size = layer_size
        self.activation = activation

        if(self.activation is 'relu'):
            self.activation = F.relu
        elif(self.activation is 'tanh'):
            self.activation = torch.tanh

        self.layers = nn.ModuleList([nn.Linear(784,self.layer_size)])
        for i in range(1, self.num_hidden_layers):
            self.layers.append(nn.Linear(self.layer_size,self.layer_size))
        self.layers.append(nn.Linear(self.layer_size,10))

    def forward(self, x):
        # converting each image into a vector
        batch_size = x.shape[0]
        x = x.reshape(batch_size,-1)
        # rest of the forward pass 
        for i in range(self.num_hidden_layers+1):
            x = self.layers[i](x)
            if(self.activation is not None):
                x = self.activation(x)
        return x

Run the following code to load the MNIST dataset. For the sake of simplicity, we do not have a validation set in this problem.

In [ ]:
import torch
from torchvision import datasets, transforms

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)

# DataLoader is a nice tool provided by PyTorch for passing training or testing examples
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1000)

Here is an example of training and testing a model:

In [ ]:
import torch.nn as nn
import torch.optim as optim
import itertools

def train(model, criterion, train_loader, optimizer, epoch):
    # Turn the model to training mode (gradients will be calculated)
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We have to call zero_grad() on the optimizer to remove gradients from the previous data pass.
        # Otherwise, the gradients will be accumulated throughout many passes.
        optimizer.zero_grad()
        # Pass in the data and obtain the output.
        # When you pass the data directly by calling model(data), the model will internally pass the data through the forward() function.
        output = model(data)
        # Compare the output and the ground truth and calculate the loss.
        loss = criterion(output, target)
        # From the calculated loss, call backward() to calculate the gradients for all the paramters in the network.
        loss.backward()
        # Update the parameters according to the gradients. 
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, criterion, test_loader):
    # Turn the model to testing mode (gradients will not be calculated)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Number of training epochs
epochs = 5
# Learning rate
lr = 0.01

# Create the model
model = mnist_network(num_hidden_layers=1,layer_size=20,activation='relu')

# Define the training and testing loss
train_criterion = nn.CrossEntropyLoss()
test_criterion = nn.CrossEntropyLoss(reduction='sum')

# Define the optimizer
# We have to specify the learning rate and the parameters that the optimizer should update during training.
# In this case, we specify that the optimizer should update all the parameters from our model.
optimizer = optim.SGD(model.parameters(),lr=lr)

for epoch in range(1, epochs + 1):
        # Training
        train(model, train_criterion, train_loader, optimizer, epoch)
        # Testing
        test(model, test_criterion, test_loader)


In [ ]:
# Conder these lines of code from above
epochs = 5
lr = 0.01
model = mnist_network(num_hidden_layers=1,layer_size=20,activation='relu')

# What will you change for each part of the problem below?

## Part a

First, we will investigate the effect of varying the size of the hidden layer. Create 3 one-hidden-layer networks with the sizes of the hidden layers being 5, 20, 50, respectively. We will call these the 5-network, the 20-network, and the 50-network. All networks should use ReLU activation.

Train the 5-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

In [ ]:
# Number of training epochs
epochs = 10
# Learning rate
lr = 0.001

# Create the model
model = mnist_network(num_hidden_layers=1,layer_size=20,activation='relu')

# Define the training and testing loss
train_criterion = nn.CrossEntropyLoss()
test_criterion = nn.CrossEntropyLoss(reduction='sum')

# Define the optimizer
# We have to specify the learning rate and the parameters that the optimizer should update during training.
# In this case, we specify that the optimizer should update all the parameters from our model.
optimizer = optim.SGD(model.parameters(),lr=lr)

for epoch in range(1, epochs + 1):
        # Training
        train(model, train_criterion, train_loader, optimizer, epoch)
        # Testing
        test(model, test_criterion, test_loader)

Test the trained 5-network on the test data. Print out the accuracy.

Train the 20-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained 20-network on the test data. Print out the accuracy.

Train the 50-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained 50-network on the test data. Print out the accuracy.

Plot the training accuracies over the epochs of the networks on the same figure (there should 3 line plots/scatter plots). 

What is your conclustion on the effect of varying the hidden layer size on the performance of a neural network trained on the MNIST dataset?

## Part b

Now, we will investigate the effect of varying the number of hidden layers. Create 3 networks with 1, 2, and 3 hidden layers, respectively. The size of all hidden layers should be 20 and the activation function is ReLU. We will call these the 1-network, the 2-network, and the 3-network.

Train the 1-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, 

---

record the current training accuracy of the network. 

Test the trained 1-network on the test data. Print out the accuracy.

Train the 2-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained 2-network on the test data. Print out the accuracy.

Train the 3-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained 3-network on the test data. Print out the accuracy.

Plot the training accuracies over the epochs of the networks on the same figure (there should 3 line plots/scatter plots). 

What is your conclustion on the effect of varying the number of hidden layers on the performance of a neural network trained on the MNIST dataset?

## Part c

Next, we will investigate the effects of varying the activation functions on a neural network. Create 3 networks. The first network has Sigmoid activation (Sigmoid-network). The second network has ReLU activation (ReLU-network). The third network has Tanh activation (Tanh-network). All networks have one hidden layer with size 20.

Train the Sigmoid-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained Sigmoid-network on the test data. Print out the accuracy.

Train the ReLU-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained ReLU-network on the test data. Print out the accuracy.

Train the Tanh-network on the MNIST dataset for 10 epochs with learning rate 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained Tanh-network on the test data. Print out the accuracy.

Plot the training accuracies over the epochs of the networks on the same figure (there should 3 line plots/scatter plots). 

What is your conclustion on the effect of varying the activation functions on the performance of a neural network trained on MNIST dataset?

## Part d

Finally, we will look into the effect of varying the value of the learning rate on the performance of a neural network. Create a network with one hidden layer of size 20 and ReLU activation.

Train the network on the MNIST dataset for 10 epochs. Set the learning rate to be 0.1. After each epoch, record the current training accuracy of the network. 

Test the trained network on the test data. Print out the accuracy.

Train the network on the MNIST dataset for 10 epochs. Set the learning rate to be 0.01. After each epoch, record the current training accuracy of the network. 

Test the trained network on the test data. Print out the accuracy.

Train the network on the MNIST dataset for 10 epochs. Set the learning rate to 0.001. After each epoch, record the current training accuracy of the network. 

Test the trained network on the test data. Print out the accuracy.

Plot the training accuracies over the epochs of the scenarios on the same figure (there should 3 line plots/scatter plots). 

What is your conclustion on the effect of varying the learning rate on the performance of a neural network?

## REMARK for Problem 2

You have observed the effects of varying different hyperparameters on the performance of a neural network **on the MNIST dataset**. However, keep in mind that these trends only apply for **the MNIST dataset** and should not be carried to another problem. There is no single hyperparameter settings that works for all problems. As you do more problems, you will build up your intuitions about the hyperparameters so that you can quickly deploy a good model. For example, people observed that setting the learning rate = 0.001 often works the best, though it is not always the case.

## Problem 3

Experimenting with **k-anomity, i-diversity, and t-closeness**. 

Consider a dataset, for example, with 3 ordinary attributes and 1 sensitive attribute. Let the 3 ordinary attributes be Age, Sex, and Education and the sensitive attribute be Income, each row in this dataset is of the form:

$$
    [Age, Sex, Education, Income]
$$

A hacker is interested in knowing the sensitive attribute Income. When the dataset is designed so that if complies with either **k-anomity**, **i-diversity**, and/or **t-closeness**, even if he or she somehow figures out the values of the three, the hacker may not retrive the sensitive information accurately. In general, **k-anomity** is weaker than **i-diversity**, which, in turn, is weaker than **t-closeness**.

By definition, **k-anomity** means that there is at least **k** different rows in the table of which ordinary values are a particular combination of Age, Sex, and Education. For example, the hacker knows the information of the person of interest is Age = 31, Sex = Female, and Education = BS. He or she looks into the data table and found that there are 3 rows with that combination:

$$
    [Age=31, Sex=Female, Education=BS, Income=300k]
$$
$$
    [Age=31, Sex=Female, Education=BS, Income=70k]
$$
$$
    [Age=31, Sex=Female, Education=BS, Income=20k]
$$

The hacker cannot tell accurately what the income of the person is because it can be one of the 3 values shown. This particular combination of information has 3-anomity. If every combination corresponds to at least 3 rows, then the dataset has 3-anomity.

a) Let's look at the dataset **"table.csv"**. Let the sensitive attribute be **education** and others be ordinary attributes. Calculate the anomity of the dataset (the value **k**). First, find all the posible combinations of the ordinary attributes that exists in the dataset. After that, determine the anomity for each combination. The anomity of the dataset is the smallest anomity among the combinations.

In [ ]:
# As you read here, k-anonyminity is calculated by looking at the combinations of rows / features  
# not including the sensitive attribute
# If we total up the number of times each combination occurs in a dataset, we can take
# the minimum value of that and that will be our k-anonymity 

We can improve the **k-anomity** of the dataset by "suppressing" the ordinary attributes. Suppressing means reducing the resolution of the attribute's value. For this problem, let's suppress Age by replacing the exact age with an age range. For example, instead of leaving age = 32, replace it with age = 30-40. Apply this to **"table.csv"** with the ranges {<20, 20-30, 30-50, >50}. Check if the anomity improves. 

In [ ]:
# Due to the the number of possible combinations with all the unique values in the last part
# the k-anonymity was probability pretty low, as you would need every combination to occur
# at LEAST k times 
# So to make the k-anonymity higher, we can suppress some of the attributes to a range
# to make the number of combinations lower and the number of each combination higher

**K-anomity** is nice, however, it fails in many cases. If the rows which share a combination of ordinary attributes have only a few values for the sensitive attribute, then it is not much better than having no anomity at all. For example, consider:

$$
    [Age=31, Sex=Female, Education=BS, Income=300k]
$$
$$
    [Age=31, Sex=Female, Education=BS, Income=20k]
$$
$$
    [Age=31, Sex=Female, Education=BS, Income=20k]
$$
$$
    [Age=31, Sex=Female, Education=BS, Income=20k]
$$

When **k-anomity** fails in the second case, **i-diversity** comes to the rescue. **I-diversity** states that the rows of a particular combination of information must have at least i different values for the sensitive attribute. The above example has 2-diversity, which is not good. 

b) Calculate the **i-diversity** of the dataset **"table.csv"**. Follow similar steps as in part a. 

In [ ]:
# K-anonymity does not really protect the material of the sensitive attribute itself though
# I-diversity is the next step of security above that
# If we look at all the combinations of attributes we generated from our k-anonymity 
# within each combination, we look at the different values of sensitive attributes
# and count how mnay unique instances of it occur
# If our i-diversity is low, ie 1, then the combination of other attributes does 
# not really protect the sensitive data, because if they know the other attributes,
# then you essentially know the last value

# To calculate i-diversity, you would go through all combinations of of your non-sensitive
# attributes, and find the number of unique sensitive attributes
# Then you would take the minimum number of this over the whole dataset

Suppressing an attribute can also improve the **i-diversity** of the dataset. Repeat the suppression as in **part a** and check if the diversity improves. If it does not, consider further suppress age by using the range {<20, 20-50, >50}.

**T-closeness** is even better than **i-diversity**. **T-closeness** requires that for every combination of information, the distribution of the sensitive attribute's value among the corresponding rows must be close to the overall distribution of the sensitive attribute's value for the whole dataset. Distance between distribution is calculated using the Earth Mover Distance (EMD). The dataset has **t-closeness** if no distance exceeds **t**. 

c) Calculate the overall distribution of **education**. Find the **t-closeness** of the dataset (largest distance between any combination's distribution of marital-status and the overall distribution).

You can use **scipy.stats.wasserstein_distance** to calculate the EMD.

In [ ]:
# We just saw with i-diversity that we do not want just a few values for every combination
# For every combination of attributes, we want to have a nice distribution 
# of sensitive values
# And ideally, we want to have this distribution of sensitive information within each 
# combination match the distribution of sensitive values for the entire dataset

# T-closeness measures how close the distribution of values within a combination is 
# to the distribution of the entire dataset

# If we find the distribution of the whole dataset, and then loop through the distributions
# of each combination, we can calculate the EMD (distance between distributions) 
# for each of the combinations
EMD = wasserstein_distance(overall_distribution,distribution)

# We are wanting to find the max distance in this problem, so we would take the 
# max value of this across all combinations 